# 04 - Forecasting Future Demand

Use the trained Ridge regression model to forecast future units sold.

In [3]:
# 📦 Imports
import pandas as pd
import numpy as np
import joblib
from datetime import datetime, timedelta

In [13]:
# 📂 Load model
ridge_model = joblib.load("../models/ridge_forecast_model.pkl")  # Replace with correct ridge model path if saved
print("✅ Model loaded")

✅ Model loaded


In [14]:
# 📅 Generate future dates
future_dates = pd.date_range(start="2025-08-01", end="2025-08-15", freq='D')
items = ['Toothpaste', 'Soap', 'Diaper']
brands = {'Toothpaste': 'Colgate', 'Soap': 'Dove', 'Diaper': 'Pampers'}
categories = {'Toothpaste': 'Personal Care', 'Soap': 'Personal Care', 'Diaper': 'Baby Care'}
price_map = {'Toothpaste': 45, 'Soap': 25, 'Diaper': 220}

# Create full future dataframe
rows = []
for date in future_dates:
    for item in items:
        rows.append({
            "date": date,
            "item": item,
            "brand": brands[item],
            "category": categories[item],
            "price": price_map[item],
            "promotion": 0,  # placeholder
            "discount": 0.0,  # placeholder
        })

future_df = pd.DataFrame(rows)

In [15]:
# 🧠 Add time features
future_df["day_of_week"] = future_df["date"].dt.dayofweek
future_df["month"] = future_df["date"].dt.month
future_df["is_weekend"] = future_df["day_of_week"].isin([5, 6]).astype(int)
future_df["is_holiday"] = 0  # Placeholder, optionally integrate real holiday API

In [16]:
# 🔤 Encode categorical features (same encoding as training)
item_map = {'Toothpaste': 0, 'Soap': 1, 'Diaper': 2}
category_map = {'Personal Care': 0, 'Baby Care': 1}
brand_map = {'Colgate': 0, 'Dove': 1, 'Pampers': 2}

future_df["item"] = future_df["item"].map(item_map)
future_df["category"] = future_df["category"].map(category_map)
future_df["brand"] = future_df["brand"].map(brand_map)

In [17]:
# 🧪 Prepare features (use only those from training)
features = ['item', 'category', 'price', 'promotion', 'discount',
            'brand', 'day_of_week', 'month', 'is_weekend', 'is_holiday']
X_future = future_df[features]

In [18]:
# show first few rows of the X_future DataFrame
X_future.head()

,item,category,price,promotion,discount,brand,day_of_week,month,is_weekend,is_holiday
0,0,0,45,0,0.0,0,4,8,0,0
1,1,0,25,0,0.0,1,4,8,0,0
2,2,1,220,0,0.0,2,4,8,0,0
3,0,0,45,0,0.0,0,5,8,1,0
4,1,0,25,0,0.0,1,5,8,1,0


In [19]:
# 🔮 Predict future demand
future_df["predicted_units_sold"] = ridge_model.predict(X_future)
future_df[["date", "item", "predicted_units_sold"]]

,date,item,predicted_units_sold
0,2025-08-01,0,37.918272
1,2025-08-01,1,37.677790
2,2025-08-01,2,32.412210
3,2025-08-02,0,37.772033
4,2025-08-02,1,37.531551
5,2025-08-02,2,32.265971
6,2025-08-03,0,37.845639
7,2025-08-03,1,37.605158
8,2025-08-03,2,32.339578
9,2025-08-04,0,37.623844


In [20]:
# 💾 Save forecast
future_df.to_csv("../data/processed/future_demand_forecast.csv", index=False)
print("✅ Forecast saved to ../data/processed/future_demand_forecast.csv")

✅ Forecast saved to ../data/processed/future_demand_forecast.csv
